In [52]:
import sys,os
sys.path.append("..")

MONGO_URL="localhost:27017"
os.environ["MONGO_URL"] = MONGO_URL

MONGO_VENDORBOT_DATABASE = "vendorbot_db"
os.environ["MONGO_VENDORBOT_DATABASE"] = MONGO_VENDORBOT_DATABASE

MOLECULES_DATABASE = "molecules_db"
os.environ["MOLECULES_DATABASE"] = MOLECULES_DATABASE

MONGO_BOT_USERNAME="bot"
os.environ["MONGO_BOT_USERNAME"] = MONGO_BOT_USERNAME

MONGO_BOT_PASSWORD="botpsswd"
os.environ["MONGO_BOT_PASSWORD"] = MONGO_BOT_PASSWORD

from modules.db.dbconfig import db_client
from modules.db.users import UsersCollection
from modules.db.unique_molecules import UniqueMolecules

# collections of data to work with
users_collection = UsersCollection(db_client, MONGO_VENDORBOT_DATABASE)
unique_molecules_collection = UniqueMolecules(db_client, MOLECULES_DATABASE)

print(db_client)
db_client.list_database_names()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', connecttimeoutms=5)


['admin',
 'blacklist_rdkit_db',
 'config',
 'local',
 'molecules_db',
 'test_db',
 'vendorbot_db']

In [53]:

user_id = 122267418
query = {"user_id": user_id}
users_collection.count_documents({"user_id": user_id}) 

AttributeError: 'UsersCollection' object has no attribute 'count_documents'

In [54]:
query = {'index': moldoc['index']}
unique_molecules_collection.count_documents({"index": index}) 

AttributeError: 'UniqueMolecules' object has no attribute 'count_documents'

In [55]:
from rdkit import Chem
##########################
#FIXING CHARGED MOLECULES#
##########################
def neutralize_atoms(mol):
    '''
    atom neutralizer
    '''
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

from rdkit.Chem.rdmolops import GetFormalCharge
def charge_fixer(SMILES):
    '''
    Neutralize molecules atom by atom
    '''
    # Create RDKit molecular objects
    mol = Chem.MolFromSmiles(smiles)
    if GetFormalCharge(mol) != 0:
        neutralize_atoms(mol)
    return Chem.MolToSmiles(mol)

In [56]:
# SMILES
smiles = 'CC[C@H]([NH3+])CO'

In [57]:
from mongordkit.Database import write
from mongordkit.Search import similarity, substructure
from mongordkit.Database import registration

In [59]:
SMILES = 'C[C@H](N)CO'
reagent_internal_id = 'a05785ae720b414ca41526baf22bc026'
user_id = 122267418
query = {"user_id": user_id}

def reagent_registration(SMILES, reagent_internal_id):

    molfile = Chem.MolFromSmiles(SMILES)
    scheme = registration.MolDocScheme()
    scheme.add_value_field('reagent_internal_id_list', [reagent_internal_id])
    moldoc = scheme.generate_mol_doc(molfile)

    result = write.WriteFromMolList(db_client[MOLECULES_DATABASE].unique_molecules_collection, [molfile], scheme=scheme) 

    # print(query)
    if result == 0:

        reagent_internal_id_list = unique_molecules_collection.get_molecule(moldoc['index'])["value_data"]['reagent_internal_id_list']
        
        if reagent_internal_id not in reagent_internal_id_list:
            reagent_internal_id_list.append(reagent_internal_id)

            scheme.add_value_field('reagent_internal_id_list', reagent_internal_id_list)

            moldoc = scheme.generate_mol_doc(molfile)
            unique_molecules_collection.update_molecule(moldoc['index'], moldoc)

    # print(insertion_result.acknowledged)

reagent_registration(SMILES, reagent_internal_id)

populating mongodb collection with compounds from list...
0 molecules successfully imported
1 duplicates skipped


In [42]:
molfile = Chem.MolFromSmiles(SMILES)
scheme = registration.MolDocScheme()
scheme.add_value_field('reagent_internal_id_list', [reagent_internal_id])
moldoc = scheme.generate_mol_doc(molfile)
print(moldoc)
unique_molecules_collection.add_molecule(moldoc)
result = write.WriteFromMolList(unique_molecules_collection, [molfile], scheme=scheme) 

{'rdmol': Binary(b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x04\x00\x00\x00\x80\x01\x06\x00`\x00\x00\x00\x01\x03\x06 4\x00\x00\x00\x02\x01\x04\x07\x00`\x00\x00\x00\x01\x02\x06\x00`\x00\x00\x00\x02\x02\x08\x00`\x00\x00\x00\x01\x01\x0b\x00\x01\x00\x01\x02\x00\x01\x03\x00\x03\x04\x00\x14\x00\x17\x04\x00\x00\x00\x00\x00\x00\x00\x16', 0), 'index': 'BKMMTJMQCTUHRP-VKHMYHEASA-N', 'smiles': 'C[C@H](N)CO', 'scheme': 'default', 'hashes': {'CanonicalSmiles': 'C[C@H](N)CO', 'inchikey_standard': 'BKMMTJMQCTUHRP-VKHMYHEASA-N'}, 'fingerprints': {}, 'value_data': {'reagent_internal_id_list': ['a05785ae720b414ca41526baf22bc026']}}
populating mongodb collection with compounds from list...


AttributeError: 'UniqueMolecules' object has no attribute 'count_documents'

In [ ]:
print(unique_molecules_collection)

In [86]:
from rdkit import RDLogger, Chem
from mongordkit.Database import write
from mongordkit.Search import similarity, substructure

from rdkit.Chem import PandasTools, SanitizeMol


# Search.PrepareForSearch(rdkit_db, rdkit_db.molecules, rdkit_db.mfp_counts, rdkit_db.permutations)
substructure.AddPatternFingerprints(unique_molecules_collection)
similarity.AddMorganFingerprints(unique_molecules_collection, db_client[MOLECULES_DATABASE].mfp_counts)

# Generate 100 different permutations of length 2048 and save them in demo_db.permutations as separate documents.
similarity.AddRandPermutations(db_client[MOLECULES_DATABASE].permutations)

# Add locality-sensitive hash values to each documents in demo_db.molecules by splitting the 100 different permutations
# in demo_db.permutations into 25 different buckets.
similarity.AddLocalityHashes(unique_molecules_collection, db_client[MOLECULES_DATABASE].permutations, 25)

# Create 25 different collections in db_demo each store a subset of hash values for molecules in demo_db.molecules.
similarity.AddHashCollections(db_client[MOLECULES_DATABASE], unique_molecules_collection)

In [ ]:
from operator import itemgetter
def similarity_search(database, smiles: str):
    """
    Ищем похожие реагенты умными функциями
    :param smiles:
    :return: отсортированный по похожести список реагентов
    """
    smiles = smiles.replace("|", "")  # вертикальная черта в SMILES - непонятно что несёт, и RDKIT ее не понимает, убираем ее
    mol = Chem.MolFromSmiles(smiles)

    if not mol:
        raise Exception("MolFromSmiles returned None")

    res = similarity.SimSearchAggregate(mol, database.unique_molecules_collection, database.mfp_counts, 0.1)

    if not res:
        return

    res = sorted(res, key=itemgetter(0), reverse=True)

    if res[0] is None:
        raise Exception(f"incorrect result: {str(res)}")

    return res

In [87]:
def get_most_similar_reagent(SMILES):
    
    best_similarity_result_id = similarity_search(db_client[MONGO_VENDORBOT_DATABASE], SMILES)[0][1]
    reagent_internal_id_list = unique_molecules_collection.get_molecule(best_similarity_result_id)["value_data"]['reagent_internal_id_list']

    return reagent_internal_id_list

SMILES = 'CCN=C=NCCCN(C)C'
get_most_similar_reagent(SMILES)
user_id = "122267418"

def batch_reagent_registration(user_id: str):
    reagent_list = users_collection.get_reagents(user_id)

    for reagent in reagent_list:
        SMILES = reagent['SMILES']
        reagent_internal_id = reagent['reagent_internal_id']
        reagent_registration(SMILES, reagent_internal_id)

In [60]:
from modules.db.dbschema import UserReagents

In [68]:
user_reagents = UserReagents(users_collection.get_user(user_id))
user_reagents.export()


{'_id': '9e9dab00e5f842ab90d156e90b618bb1',
 'args': ({'_id': 122267418,
   'user_id': 122267418,
   'username': 'radical_subject',
   'firstname': 'Oleg',
   'lastname': 'Fedorov',
   'phone_number': '',
   'user_reagents': [{'reagent_internal_id': 'd4bfb66beacb43cda65827b3ad458dc0',
     'CAS': '3849-21-6',
     'SMILES': 'CCOC(=O)/C(C#N)=N/O',
     'sharing_status': 'shared',
     'timestamp': '20.09.2022 14:58'},
    {'reagent_internal_id': '284519ce7d644cd6ae3a167ae075f01e',
     'CAS': '153433-21-7',
     'SMILES': 'CCOC(=O)/C(C#N)=N/O[P+](N1CCCC1)(N1CCCC1)N1CCCC1.F[P-](F)(F)(F)(F)F',
     'sharing_status': 'shared',
     'timestamp': '20.09.2022 14:58'},
    {'reagent_internal_id': 'e81f36465b9b484fa799ad72eb2895d0',
     'CAS': '158014-03-0',
     'SMILES': 'CCOC(=O)/C(C#N)=N/[O-].[K+]',
     'sharing_status': 'shared',
     'timestamp': '20.09.2022 14:58'},
    {'reagent_internal_id': '9316cce78f854b80989178185aac1621',
     'CAS': '958029-37-3',
     'SMILES': 'C[N+](C)=C(N1C

In [85]:
reagent_internal_id = 'd4bfb66beacb43cda65827b3ad458dc0'
if 'user_reagents' in user_reagents.export().keys():
    for reagent in user_reagents.user_reagents:
        print("check")
        if reagent_internal_id in reagent.values():
            reagent['test'] = "SUCCESS"
            print("reagent")
else:
    print('unsuc')

unsuc


In [88]:
user_reagents.export().keys()

dict_keys(['_id', 'args'])